# Predictions of optimal courier number

### Using XGBoost classifier for each unique area

### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import xgboost as xgb

import imblearn
from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler

import holidays as hld
import json
import requests as rq
from tqdm import tqdm
from datetime import datetime, timedelta
from collections import Counter

# from xgboost import XGBClassifier

import warnings
warnings.simplefilter("ignore")

C:\Users\Asus\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Function for creation merged dataframe of orders and delays for each hour
* Frequency: hour
* Linear interpolation
* Predictions from task 1 where ceiled
* var orders per courier added -- 'perc'
* values where mapped to meet the binary requirements
* values of 'perc' larger than 3.5 were considered outliers and dropped

In [2]:
def expand_df(orders, delays):
    #change types of vars
    orders['date'] = pd.to_datetime(orders['date'])
    delays['dttm'] = pd.to_datetime(delays['dttm'])
    
    #set_index of dates
    orders = orders.set_index('date')
    delays = delays.set_index('dttm')
    
    #change frequency to hour and interpolate
    for area in orders_strt.delivery_area_id.unique():
        orders[orders['delivery_area_id']==area] = orders[orders['delivery_area_id']==area].asfreq('H')
        delays[delays['delivery_area_id']==area] = delays[delays['delivery_area_id']==area].asfreq('H')

        orders[orders['delivery_area_id']==area] = orders[orders['delivery_area_id']==area].interpolate()
        delays[delays['delivery_area_id']==area] = delays[delays['delivery_area_id']==area].interpolate()
        
    #merge by area and date
    dfs = []
    for area in orders_strt.delivery_area_id.unique():
        orders_area = orders[orders['delivery_area_id']==area].copy()
        delays_area = delays[delays['delivery_area_id']==area].copy()
        
        dfs.append(pd.merge(delays_area, orders_area, left_index=True, right_index=True, how='left'))
    con = pd.concat(dfs)
    
    #drop extra columns
    con = con.drop(columns=['delivery_area_id_y'])
    
    #interpolate and ceil NaN after merging
    con['orders_cnt'] = con['orders_cnt'].interpolate().apply(np.ceil)
    
    #drop NaN (there are only one missing value)
    con = con.dropna()
    
    #create orders per courier
    con['perc'] = con['orders_cnt']/con['partners_cnt']
    
    #rename to better reflect the idea
    con = con.rename(columns={'delay_rate': 'delay'})
    
    #turn values to 0 and 1
    con['delay'] = (con['delay']>0.05)*1
    
    #drop outliers
    con = con[con['perc']<3.5]
    
    return con

In [3]:
def choose_region(df, area):
    #get values of particular area
    return df[df['delivery_area_id_x']==area]

## Feature creation
Added:
* day of week
* hour
* weekday
* day_of_year
* month
* average particular day of month frequency in all observations
* average particular day of week frequency in all observations
* days until nearest holiday
* days after nearest holiday
* days until nearest non-working day
* days after nearest non-working day
* if a day is holiday
* if a day is weekend

In [4]:
def create_features(delays):
    
    delays = delays.reset_index()
    delays['dttm'] = pd.to_datetime(delays['dttm'])
    delays_dates = delays['dttm']
    delays['day'] = delays_dates.apply(lambda x: x.day)
    delays['hour'] = delays_dates.apply(lambda x: x.hour)
    delays['weekday'] = delays_dates.apply(lambda x: x.weekday())
    delays['day_of_year'] = delays_dates.apply(lambda x: x.dayofyear)
    delays['month'] = delays['day_of_year'] = delays_dates.apply(lambda x: x.month)
    day_freq_delays = delays['day'].value_counts(normalize=True).to_dict()
    delays['day_freq'] = delays['day'].map(day_freq_delays)
    weekday_freq_delays = delays['weekday'].value_counts(normalize=True).to_dict()
    delays['weekday_freq'] = delays['weekday'].map(weekday_freq_delays)
    
    
    delays_dates = delays['dttm']
    
    holidays = [date[0] for date in hld.Russia(years=2021).items()]

    delays_dates = delays_dates.to_frame().sort_values(by='dttm')

    df_holidays = pd.DataFrame({'holidays': holidays})
    df_holidays['holidays'] = pd.to_datetime(df_holidays['holidays'])

    delays_dates = pd.merge_asof(delays_dates, df_holidays, left_on='dttm', right_on='holidays', direction='forward')
    delays_dates = pd.merge_asof(delays_dates, df_holidays, left_on='dttm', right_on='holidays')

    delays_dates['days_until_holiday'] = delays_dates.pop('holidays_x').sub(delays_dates['dttm']).dt.days
    delays_dates['days_since_holiday'] = delays_dates['dttm'].sub(delays_dates.pop('holidays_y')).dt.days

    delays_dates =  delays_dates.drop_duplicates()
    delays = pd.merge(delays, delays_dates, how='left', on='dttm')
    delays_dates = delays['dttm']

    url = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/consultant2021.json'
    response = rq.get(url)
    non_working = json.loads(response.text)

    for days in non_working:
        non_working[days] = pd.to_datetime(non_working.get(days)).date

    delays['is_holiday'] = delays_dates.apply(lambda x: x.date() in non_working['holidays']) * 1
    delays['is_weekend'] = ((delays['weekday']==5) | (delays['weekday']==6)) * 1
    delays_dates = delays_dates.to_frame().sort_values(by='dttm')

    sorted_nonworking = np.sort(np.concatenate((non_working['holidays'], non_working['nowork'])))

    df_non_working = pd.DataFrame({'non_working': sorted_nonworking})
    df_non_working['non_working'] = pd.to_datetime(df_non_working['non_working'])

    delays_dates = pd.merge_asof(delays_dates, df_non_working, left_on='dttm', right_on='non_working', direction='forward')
    delays_dates = pd.merge_asof(delays_dates, df_non_working, left_on='dttm', right_on='non_working')

    delays_dates['days_until_nonworking'] = delays_dates.pop('non_working_x').sub(delays_dates['dttm']).dt.days
    delays_dates['days_since_nonworking'] = delays_dates['dttm'].sub(delays_dates.pop('non_working_y')).dt.days

    delays_dates =  delays_dates.drop_duplicates()

    delays = pd.merge(delays, delays_dates, how='left', on='dttm')
    
    delays = delays.set_index('dttm')

    return delays

## Oversampling

Oversample given observation with synthetic data to get 50/50 target class distribution

In [5]:
def oversampling(df, TARGET):
    #Get the negative to positive weight class estimation
    def est(df, TARGET):
        counter = Counter(df[TARGET])
        # estimate scale_pos_weight value
        return counter[0] / counter[1]
        
    print(f"Initial negative to positive classes weight disbalance: {est(df, TARGET):.3f}")
    
    ### PREVIOUSLY USED TO UNIFORMLY DISTRIBUTE COURIERS ACROSS DATA FOR BETTER CLASSIFICATION
    ### UNSUCCESFUL ATTEMPT 
    # con['partners_cnt'] = con['partners_cnt'].apply(round)
    # sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    # # Fit the model to generate the data.
    # oversampled_trainX, oversampled_trainY = sampler.fit_resample(con.drop('partners_cnt', axis=1), con['partners_cnt'])
    # oversampled_train = pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis=1)
    # con = oversampled_train.copy()
    
    sm = SMOTE(sampling_strategy='auto', random_state=42)

    # Fit the model to generate the data.
    oversampled_trainX, oversampled_trainY = sm.fit_resample(df.drop(TARGET, axis=1), (df[TARGET]>0.05)*1)
    oversampled_train = pd.concat([pd.DataFrame(oversampled_trainY), pd.DataFrame(oversampled_trainX)], axis=1)
    
    print(f"Final negative to positive classes weight disbalance: {est(oversampled_train, TARGET):.3f}")
    
    return oversampled_train

In [6]:
def recreate_dates(df):
    #after resampling, the date index values are dropped and chanded to range index, here we recreate dates
    
    dt = lambda x: '0' + str(x) if len(str(x))==1 else str(x)
    df['month'] = df['month'].apply(dt)
    df['day'] = df['day'].apply(dt)
    df['hour'] = df['hour'].apply(dt)
    
    df['dttm'] = '2021'+'-'+df['month']+'-'+df['day']+' '+df['hour']
    tr_dates = lambda x: datetime.strptime(x, "%Y-%m-%d %H")
    df['dttm'] = df['dttm'].apply(tr_dates)
    df = df.set_index('dttm')
    
    df['month'] = df['month'].astype('int')
    df['day'] = df['day'].astype('int')
    df['hour'] = df['hour'].astype('int')
    
    
    return df

In [7]:
def print_scores(model, X_val, y_val):
    #function for printing scores during the model training

    y_pred = reg.predict(X_val)
    
    print(f"Accuracy: {accuracy_score((y_val>0.05)*1, y_pred):.3f}")
    
    try:
        print(f"ROC-AUC: {roc_auc_score((y_val>0.05)*1, y_pred):.3f}")
    except ValueError:
        pass
    
    print(f"f1: {f1_score((y_val>0.05)*1, y_pred):.3f}")

## Function for varying couriers
Vary couriers number to get different values for delay and find the optimal num of couriers afterwards

In [8]:
def vary_couriers(df, model, FEATURES):
    
    df_proxy = df.copy()
    df_lst = []
    for i in range(len(df_proxy)):
        proxy = pd.DataFrame([df_proxy.iloc[i]]*20)
        proxy["partners_cnt"] = range(1, 21)
        df_lst.append(proxy)
    
    df_exp = pd.concat(df_lst)
    df_exp['perc'] = df_exp['orders_cnt'] / df_exp['partners_cnt']
    
    if 'delay' in df_exp.columns:
        df_exp = df_exp.drop('delay', axis=1)
    
    preds = reg.predict(df_exp[FEATURES])

    df_varied = pd.concat([df_exp.reset_index(), pd.DataFrame(preds, columns=['delay'])], axis=1)
    
    return df_varied

## Function for predictions
Choose the best prediction to solve the task

As some predictions from varying the couriers might be definitely wrong (data distribution problems), function makes empirically received border for delay, so that we consider the 'perc' values higher than 2.5 as the definite delay. After that we sort values and get the best prediction that has lowest delay (0 or 1), num of couriers ('partners_cnt') and 'perc'.

If the model predicted that there would be only delays for particular datetime, function returns the result, closest to perc ~ 2, as we consider such delay predictions as a data distribution problems

In [56]:
def choose_best_pred(con_cls_proxy):
    con_cls_proxy.loc[con_cls_proxy['perc']>2.5, 'delay'] = 1 
    
    predictions = []
    for i in con_cls_proxy['index'].unique():
        prx = con_cls_proxy[con_cls_proxy['index']==i].sort_values(by=['delay', 'partners_cnt', 'perc'])

        if prx['delay'].iloc[0] == 1:

            prx = prx.iloc[(prx['perc'] - 2).abs().argsort(),:]

        prediction = prx.iloc[[0]]
        predictions.append(prediction)
    
    return pd.concat(predictions)

In [10]:
orders_pred = pd.read_csv('final_res.csv')
orders_strt = pd.read_csv('orders.csv')
delays_strt = pd.read_csv('partners_delays.csv')

In [11]:
orders = orders_strt.copy()
delays = delays_strt.copy()

In [12]:
con_exp = expand_df(orders, delays)

In [14]:
areas = con_exp['delivery_area_id_x'].unique()

In [15]:
FEATURES = ['partners_cnt', 'orders_cnt', 
            'perc', 'month', 'day', 'hour', 'weekday', 
            'day_of_year', 'day_freq', 'weekday_freq',
            'days_until_holiday', 'days_since_holiday', 'is_holiday', 'is_weekend',
            'days_until_nonworking', 'days_since_nonworking']

# FEATURES = ['partners_cnt', 'orders_cnt', 'perc',]
TARGET = 'delay'

In [23]:
%%time

#dicts for collecting historical model data 
area_pred = dict()
area_accuracy = dict()
area_losses = dict()
area_roc_auc = dict()
area_f1 = dict()

#iterate through all areas
for area in areas:
    preds = []
    accuracy = []
    roc_auc = []
    f1 = []
    losses = []
    
    con = choose_region(con_exp, area)    #choosing particular region
    con = create_features(con)    #creating features for the region
    
    #splitting data to get testing X_val, y_val before oversampling
    X_train, X_val, y_train, y_val = train_test_split(con[FEATURES], con[TARGET], test_size=.05)    
    
    con = pd.concat([X_train, y_train], axis=1)    #concat train data
    con = oversampling(con, TARGET)    #oversample to meet 50/50 class distribution
    con = recreate_dates(con)    #recreate dates
    con['perc'] = con['orders_cnt']/con['partners_cnt']    #recalculate percs after all manipulations
    
    #params for classifier, n_estimators chooses depending on the num of observations in order to prevent overfitting
    params_cls = {'base_score': 0.5,
     'booster': 'gbtree',
     'learning_rate': 0.01,
     'max_depth': 3,
     'n_estimators': int(len(con)*.2),
#      'early_stopping_rounds': 50,
     'objective':'binary:logistic',
     'eval_metric': 'logloss',
     'use_label_encoder': False,
     'n_jobs': -1
    }
    
    #split timeseries
    tscv = TimeSeriesSplit(n_splits=5, test_size=int(len(con)*.05))
    
    print(f"##########################################area_{area}############################################")
    for train_index, test_index in tscv.split(con):

        train = con.iloc[train_index]
        test = con.iloc[test_index]

        X_train_iter = train[FEATURES]
        y_train_iter = train[TARGET]

        X_test = test[FEATURES]
        y_test = test[TARGET]

        reg = xgb.XGBClassifier(**params_cls)
        
        #train and eval
        reg.fit(X_train_iter, y_train_iter,
            eval_set=[(X_train_iter, y_train_iter), (X_test, y_test)],
            verbose=10)
        
        #calc preds and probs
        y_pred = reg.predict(X_test)
        y_pred_proba = reg.predict_proba(X_test)
        
        #add to historical data
        preds.append(y_pred)
        
        losses.append(log_loss(y_test, y_pred, labels=[0, 1]))
        accuracy.append(accuracy_score(y_test, y_pred))
        
        try:
            roc_auc.append(roc_auc_score(y_test, y_pred))
        except ValueError:
            pass
        
        f1.append(f1_score(y_test, y_pred))
        
        print("Iteration scores:")
        print_scores(reg, X_val, y_val)
        print("-----------------------")
                  
    #after training the model on particular area, save the weights              
    reg.save_model(f'..\\courier hackathon\\weights\\area_{area}.json')
    
    #write historical data
    area_pred[area] = preds
    area_losses[area] = losses
    area_roc_auc[area] = roc_auc
    area_f1[area] = f1
    area_accuracy[area] = accuracy

Initial negative to positive classes weight disbalance: 14.264
Final negative to positive classes weight disbalance: 1.000
##########################################area_443############################################
[0]	validation_0-logloss:0.68633	validation_1-logloss:0.68734
[10]	validation_0-logloss:0.62492	validation_1-logloss:0.63489
[20]	validation_0-logloss:0.57380	validation_1-logloss:0.59110
[30]	validation_0-logloss:0.53077	validation_1-logloss:0.55425
[40]	validation_0-logloss:0.49422	validation_1-logloss:0.52298
[50]	validation_0-logloss:0.46278	validation_1-logloss:0.49454
[60]	validation_0-logloss:0.43586	validation_1-logloss:0.46956
[70]	validation_0-logloss:0.41328	validation_1-logloss:0.44832
[80]	validation_0-logloss:0.39371	validation_1-logloss:0.42975
[90]	validation_0-logloss:0.37664	validation_1-logloss:0.41403
[100]	validation_0-logloss:0.36148	validation_1-logloss:0.39964
[110]	validation_0-logloss:0.34746	validation_1-logloss:0.38612
[120]	validation_0-loglos

[600]	validation_0-logloss:0.17935	validation_1-logloss:0.19007
[610]	validation_0-logloss:0.17820	validation_1-logloss:0.18854
[620]	validation_0-logloss:0.17693	validation_1-logloss:0.18710
[626]	validation_0-logloss:0.17616	validation_1-logloss:0.18629
Iteration scores:
Accuracy: 0.876
ROC-AUC: 0.672
f1: 0.353
-----------------------
[0]	validation_0-logloss:0.68639	validation_1-logloss:0.68620
[10]	validation_0-logloss:0.62557	validation_1-logloss:0.62351
[20]	validation_0-logloss:0.57499	validation_1-logloss:0.57125
[30]	validation_0-logloss:0.53246	validation_1-logloss:0.52720
[40]	validation_0-logloss:0.49643	validation_1-logloss:0.49001
[50]	validation_0-logloss:0.46617	validation_1-logloss:0.45855
[60]	validation_0-logloss:0.44013	validation_1-logloss:0.43142
[70]	validation_0-logloss:0.41744	validation_1-logloss:0.40714
[80]	validation_0-logloss:0.39726	validation_1-logloss:0.38558
[90]	validation_0-logloss:0.37937	validation_1-logloss:0.36651
[100]	validation_0-logloss:0.363

[570]	validation_0-logloss:0.17653	validation_1-logloss:0.15970
[580]	validation_0-logloss:0.17526	validation_1-logloss:0.15827
[590]	validation_0-logloss:0.17418	validation_1-logloss:0.15709
[600]	validation_0-logloss:0.17290	validation_1-logloss:0.15564
[610]	validation_0-logloss:0.17167	validation_1-logloss:0.15435
[620]	validation_0-logloss:0.17048	validation_1-logloss:0.15307
[626]	validation_0-logloss:0.16981	validation_1-logloss:0.15224
Iteration scores:
Accuracy: 0.888
ROC-AUC: 0.743
f1: 0.444
-----------------------
[0]	validation_0-logloss:0.68636	validation_1-logloss:0.68658
[10]	validation_0-logloss:0.62522	validation_1-logloss:0.62761
[20]	validation_0-logloss:0.57439	validation_1-logloss:0.57881
[30]	validation_0-logloss:0.53229	validation_1-logloss:0.53721
[40]	validation_0-logloss:0.49665	validation_1-logloss:0.50162
[50]	validation_0-logloss:0.46609	validation_1-logloss:0.47069
[60]	validation_0-logloss:0.43962	validation_1-logloss:0.44388
[70]	validation_0-logloss:0.4

[520]	validation_0-logloss:0.12333	validation_1-logloss:0.16362
[530]	validation_0-logloss:0.12198	validation_1-logloss:0.16143
[540]	validation_0-logloss:0.12079	validation_1-logloss:0.15961
[550]	validation_0-logloss:0.11943	validation_1-logloss:0.15727
[560]	validation_0-logloss:0.11810	validation_1-logloss:0.15511
[570]	validation_0-logloss:0.11677	validation_1-logloss:0.15285
[580]	validation_0-logloss:0.11563	validation_1-logloss:0.15096
[590]	validation_0-logloss:0.11455	validation_1-logloss:0.14908
[600]	validation_0-logloss:0.11349	validation_1-logloss:0.14718
[610]	validation_0-logloss:0.11252	validation_1-logloss:0.14543
[620]	validation_0-logloss:0.11135	validation_1-logloss:0.14344
[630]	validation_0-logloss:0.11019	validation_1-logloss:0.14114
[640]	validation_0-logloss:0.10911	validation_1-logloss:0.13931
[650]	validation_0-logloss:0.10821	validation_1-logloss:0.13770
[660]	validation_0-logloss:0.10722	validation_1-logloss:0.13591
[670]	validation_0-logloss:0.10615	valid

[380]	validation_0-logloss:0.15454	validation_1-logloss:0.17493
[390]	validation_0-logloss:0.15196	validation_1-logloss:0.17218
[400]	validation_0-logloss:0.14925	validation_1-logloss:0.16918
[410]	validation_0-logloss:0.14716	validation_1-logloss:0.16705
[420]	validation_0-logloss:0.14503	validation_1-logloss:0.16459
[430]	validation_0-logloss:0.14292	validation_1-logloss:0.16246
[440]	validation_0-logloss:0.14089	validation_1-logloss:0.16024
[450]	validation_0-logloss:0.13897	validation_1-logloss:0.15825
[460]	validation_0-logloss:0.13714	validation_1-logloss:0.15608
[470]	validation_0-logloss:0.13507	validation_1-logloss:0.15332
[480]	validation_0-logloss:0.13308	validation_1-logloss:0.15070
[490]	validation_0-logloss:0.13094	validation_1-logloss:0.14768
[500]	validation_0-logloss:0.12905	validation_1-logloss:0.14507
[510]	validation_0-logloss:0.12731	validation_1-logloss:0.14271
[520]	validation_0-logloss:0.12557	validation_1-logloss:0.14034
[530]	validation_0-logloss:0.12373	valid

...

[310]	validation_0-logloss:0.21142	validation_1-logloss:0.18703
[320]	validation_0-logloss:0.20799	validation_1-logloss:0.18397
[330]	validation_0-logloss:0.20408	validation_1-logloss:0.18010
[340]	validation_0-logloss:0.20054	validation_1-logloss:0.17672
[350]	validation_0-logloss:0.19687	validation_1-logloss:0.17309
[360]	validation_0-logloss:0.19326	validation_1-logloss:0.16947
[370]	validation_0-logloss:0.18998	validation_1-logloss:0.16618
[380]	validation_0-logloss:0.18703	validation_1-logloss:0.16332
[390]	validation_0-logloss:0.18402	validation_1-logloss:0.16007
[400]	validation_0-logloss:0.18175	validation_1-logloss:0.15777
[410]	validation_0-logloss:0.17928	validation_1-logloss:0.15516
[420]	validation_0-logloss:0.17674	validation_1-logloss:0.15244
[430]	validation_0-logloss:0.17441	validation_1-logloss:0.14994
[440]	validation_0-logloss:0.17217	validation_1-logloss:0.14760
[450]	validation_0-logloss:0.16980	validation_1-logloss:0.14491
[460]	validation_0-logloss:0.16754	valid

[120]	validation_0-logloss:0.41206	validation_1-logloss:0.47141
[130]	validation_0-logloss:0.40128	validation_1-logloss:0.46187
[140]	validation_0-logloss:0.39073	validation_1-logloss:0.45134
[143]	validation_0-logloss:0.38768	validation_1-logloss:0.44879
Iteration scores:
Accuracy: 0.909
ROC-AUC: 0.952
f1: 0.500
-----------------------
Initial negative to positive classes weight disbalance: 8.317
Final negative to positive classes weight disbalance: 1.000
##########################################area_592############################################
[0]	validation_0-logloss:0.68743	validation_1-logloss:0.68896
[10]	validation_0-logloss:0.63567	validation_1-logloss:0.65052
[20]	validation_0-logloss:0.59218	validation_1-logloss:0.61611
[30]	validation_0-logloss:0.55533	validation_1-logloss:0.58643
[40]	validation_0-logloss:0.52403	validation_1-logloss:0.56053
[50]	validation_0-logloss:0.49708	validation_1-logloss:0.53786
[60]	validation_0-logloss:0.47401	validation_1-logloss:0.51747
[70]

[190]	validation_0-logloss:0.33128	validation_1-logloss:0.33455
[200]	validation_0-logloss:0.32669	validation_1-logloss:0.32931
[210]	validation_0-logloss:0.32199	validation_1-logloss:0.32404
[220]	validation_0-logloss:0.31716	validation_1-logloss:0.31806
[230]	validation_0-logloss:0.31297	validation_1-logloss:0.31347
[240]	validation_0-logloss:0.30908	validation_1-logloss:0.30891
[250]	validation_0-logloss:0.30544	validation_1-logloss:0.30449
[260]	validation_0-logloss:0.30192	validation_1-logloss:0.29927
[270]	validation_0-logloss:0.29872	validation_1-logloss:0.29444
[280]	validation_0-logloss:0.29571	validation_1-logloss:0.29034
[290]	validation_0-logloss:0.29259	validation_1-logloss:0.28649
[300]	validation_0-logloss:0.28922	validation_1-logloss:0.28208
[310]	validation_0-logloss:0.28638	validation_1-logloss:0.27887
[320]	validation_0-logloss:0.28358	validation_1-logloss:0.27528
[330]	validation_0-logloss:0.28084	validation_1-logloss:0.27204
[340]	validation_0-logloss:0.27832	valid

[470]	validation_0-logloss:0.25120	validation_1-logloss:0.21255
[480]	validation_0-logloss:0.24988	validation_1-logloss:0.21110
[490]	validation_0-logloss:0.24863	validation_1-logloss:0.20963
[500]	validation_0-logloss:0.24705	validation_1-logloss:0.20741
[510]	validation_0-logloss:0.24566	validation_1-logloss:0.20564
[520]	validation_0-logloss:0.24423	validation_1-logloss:0.20364
[530]	validation_0-logloss:0.24316	validation_1-logloss:0.20234
[540]	validation_0-logloss:0.24191	validation_1-logloss:0.20064
[550]	validation_0-logloss:0.24060	validation_1-logloss:0.19848
[560]	validation_0-logloss:0.23916	validation_1-logloss:0.19666
[570]	validation_0-logloss:0.23719	validation_1-logloss:0.19373
[580]	validation_0-logloss:0.23584	validation_1-logloss:0.19184
[590]	validation_0-logloss:0.23451	validation_1-logloss:0.19000
[600]	validation_0-logloss:0.23301	validation_1-logloss:0.18789
[610]	validation_0-logloss:0.23183	validation_1-logloss:0.18617
[620]	validation_0-logloss:0.23050	valid

[750]	validation_0-logloss:0.21092	validation_1-logloss:0.15313
[760]	validation_0-logloss:0.20996	validation_1-logloss:0.15192
[770]	validation_0-logloss:0.20915	validation_1-logloss:0.15089
[780]	validation_0-logloss:0.20829	validation_1-logloss:0.14980
[790]	validation_0-logloss:0.20732	validation_1-logloss:0.14833
[800]	validation_0-logloss:0.20650	validation_1-logloss:0.14719
[810]	validation_0-logloss:0.20576	validation_1-logloss:0.14622
[820]	validation_0-logloss:0.20499	validation_1-logloss:0.14518
[830]	validation_0-logloss:0.20431	validation_1-logloss:0.14422
[840]	validation_0-logloss:0.20335	validation_1-logloss:0.14261
[850]	validation_0-logloss:0.20268	validation_1-logloss:0.14177
[860]	validation_0-logloss:0.20190	validation_1-logloss:0.14056
[870]	validation_0-logloss:0.20108	validation_1-logloss:0.13940
[880]	validation_0-logloss:0.20046	validation_1-logloss:0.13856
[890]	validation_0-logloss:0.19967	validation_1-logloss:0.13740
[900]	validation_0-logloss:0.19898	valid

In [69]:
# with open(f"history.json", "w") as outfile:
#     json.dump(history, outfile)

In [70]:
# for dt in history.keys():
#     for zone in history['dt']:
        

In [71]:
# history = {
#     'preds': area_pred,
#     'losses': area_losses,
#     'roc_auc': area_roc_auc,
#     'f1': area_f1,
#     'accuracy': area_accuracy
# }

# for key in history:
#     print(key)
#     with open(f"{key}.json", "w") as outfile:
#         json.dump(history[key], outfile)

In [57]:
#make predictions by loading the weights

preds_by_area = []
from tqdm import tqdm

for area in tqdm(areas):
    
    con = orders_pred[['date', f'area_id_{area}']].rename(columns={'date': 'dttm', f'area_id_{area}': 'orders_cnt'})
    con['orders_cnt'] = con['orders_cnt'].apply(np.ceil)
    reg = xgb.XGBClassifier()
    reg.load_model(f'..\\courier hackathon\\weights\\area_{area}.json')
    con = create_features(con)
    con_var = vary_couriers(con, reg, FEATURES)
    predictions = choose_best_pred(con_var)['partners_cnt'].values
    
    preds_by_area.append(pd.Series(predictions, name=f'area_id_{area}'))
    

100%|██████████| 593/593 [04:43<00:00,  2.09it/s]


In [58]:
dates = orders_pred['date'].copy()
final_couriers = pd.concat(preds_by_area, axis=1)
final_couriers.index = dates
final_couriers

,area_id_0,area_id_1,area_id_2,area_id_3,area_id_4,area_id_5,area_id_6,area_id_7,area_id_8,area_id_9,...,area_id_583,area_id_584,area_id_585,area_id_586,area_id_587,area_id_588,area_id_589,area_id_590,area_id_591,area_id_592
date,,,,,,,,,,,,,,,,,,,,,
2021-12-01 10:00:00,11,9,2,5,9,11,11,7,11,9,...,3,10,6,12,3,3,3,2,4,5
2021-12-01 11:00:00,8,7,2,4,12,11,10,13,10,6,...,3,4,6,12,3,3,3,2,3,7
2021-12-01 12:00:00,8,7,2,4,12,9,9,13,12,7,...,3,4,6,12,3,3,3,1,3,7
2021-12-01 13:00:00,8,7,2,10,12,10,9,7,10,7,...,3,10,6,12,3,3,2,2,3,3
2021-12-01 14:00:00,8,7,2,4,12,10,9,6,9,6,...,3,4,6,13,3,3,2,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-07 16:00:00,8,18,3,11,11,10,13,6,9,6,...,2,8,6,13,3,2,2,2,4,5
2021-12-07 17:00:00,8,18,3,7,14,9,13,6,10,7,...,3,3,6,5,3,2,2,2,4,5
2021-12-07 18:00:00,9,6,3,12,12,9,13,6,10,7,...,3,3,6,5,3,2,2,2,4,5


In [59]:
final_couriers.to_csv('couriers_pred.csv')

-----